# Portfolio optimization

We are now ready to work on our first portfolio optimization problem. Let us set the ground for our optimization problem.

We have a number of assets $n$ to chose from with a budget $b$, smaller than $n$ of course ($n \gt b$). This asset selection should retrieve the biggest outcome according to individual expectancies ($\mu$) and the factor of the combination between two assets (two by two in our case) is interpreted as the risk ($\sigma$). We must think of our solution as a binary vector $x \in X$ of length $n$ encoding the asset selection.

Thus, our optimization problem can be described as:
$$
\max \sum_i \mu_ix_i - \sum_i \sum_j \sigma_{i,j}x_ix_j \\
s.t. \sum_i x_i \lt b
$$

This can be easilly converted to a QUBO problem (Quadratic Unconstrained Binary Optimization) by rewritting the constraints as arguments part of the cost function. But let us start simple, shall we?

We will start witha risk free approach, only focusing on the revenew. Let us choose a set of assets to evaluate just like on previous notebooks.

In [1]:
from quanvia.finance import utils

asset_list = ['BNBUSDT','BTCUSDT','ETHUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','DOGEUSDT']
df = utils.get_binance_data(asset_list)
df.head()

,Asset,Open time,Open,High,Low,Close,Volume,Closing time,Quote asset vol,Num traders,Taker buy base asset vol,Taker buy quote asset vol,To be ignored
0,BNBUSDT,1610064000000,43.57280000,43.72200000,40.23130000,42.35600000,3548923.78800000,1610150399999,149951576.85134530,411558,1804018.63200000,76246982.43869240,0
1,BNBUSDT,1610150400000,42.34500000,44.05520000,41.50000000,43.84790000,2720363.63600000,1610236799999,116290473.53861750,294683,1458819.25300000,62429021.61184670,0
2,BNBUSDT,1610236800000,43.84790000,45.16200000,40.00000000,42.40310000,4277406.29000000,1610323199999,185165251.13206460,431771,2147084.16200000,93038871.70609660,0
3,BNBUSDT,1610323200000,42.40330000,42.50940000,35.03740000,38.16740000,6332801.05500000,1610409599999,243017347.96947130,664128,3174405.76900000,121735208.64152900,0
4,BNBUSDT,1610409600000,38.16230000,40.19890000,37.00000000,38.25410000,3261261.81000000,1610495999999,125897888.04795700,342953,1694961.86800000,65468036.38605400,0


In [2]:
exp_returns, sigma = utils.get_exp_cov(df)
# Convert expected return values to a list
mu_vals = list(exp_returns.values())

We could just encode each solution and compute their cost...

In [3]:
def binarize(num, length):
    X_val = [int(x) for x in list('{0:0b}'.format(num))]
    if len(X_val) < length:
        zero_pad = length - len(X_val)
        X_val = [0]*zero_pad + X_val
    return X_val

In [5]:
%%time

# Set the budget
budget = 2
mask = len(asset_list)

# Init solution holders
max_val = 0
max_sol = 0

# Iterate
for i in range(2**mask):
    x_sol = binarize(i, mask)
    # Solution must be valid
    if sum(x_sol) <= budget:
        val = 0
        for j,k in enumerate(x_sol):
            val += k*mu_vals[j]
        
        # Check improvement
        if val > max_val:
            max_val = val
            max_sol = i        

# Print solution
print(f'Value {max_val} for solution {bin(max_sol)}')

Value 0.033824723338734264 for solution 0b10001
Wall time: 2 ms


First and last assets seem to be the ones that will provide us bigger proffit during the day. Although this brute force method does not seem to scale well enought if we go for a larger asset list.

In [6]:
# Get first 25 elements of USDT list
asset_list = utils.get_binance_assets()
asset_list = [x for x in asset_list if x.endswith("USDT")]
asset_list = asset_list[:25]

In [7]:
2**len(asset_list)

33554432

This means we would need to check for all potential combinations from 1 to 33554432!

In [8]:
df = utils.get_binance_data(asset_list)
mu, sigma = utils.get_exp_cov(df)
# Convert expected return values to a list
mu_vals = list(mu.values())

In [9]:
%%time

# Set the budget
budget = 2
mask = len(mu)

# Init solution holders
max_val = 0
max_sol = 0

# Iterate
for i in range(2**mask):
    x_sol = binarize(i, mask)
    # Solution must be valid
    if sum(x_sol) <= budget:
        val = 0
        for j,k in enumerate(x_sol):
            val += k*mu_vals[j]
            
        # Check improvement
        if val > max_val:
            max_val = val
            max_sol = i        

print(f'Value {max_val} for solution {bin(max_sol)}')

Value 0.01625256073117966 for solution 0b100000000000100000
Wall time: 14.6 s


This is where optimization techniques come handy as they allow us to find a solution (or even **the** solution) without traversing the whole solution space. CVX is a pretty well known optimization suite for convex optimization problems. Let's give it a try!

In [10]:
import cvxpy as cp
import numpy as np

# Our solution variable
x_val = cp.Variable(len(mu), boolean=True)
rets = np.array(mu_vals)
ret = rets.T@x_val
prob = cp.Problem(cp.Maximize(ret), 
               [cp.sum(x_val) == budget, 
                x_val >= 0])

In [12]:
%%time

prob.solve(solver=cp.GLPK_MI)

Wall time: 29 ms


0.01625256073117966

In [13]:
x_val.value

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0.])

Well, there is a big difference in timming for sure!